In [26]:
from ipywidgets import (Text, Password, IntText, Button, HBox, VBox, Output, Dropdown, Label)
from IPython.display import display, clear_output
from google.colab import drive, output
import sqlite3


output.enable_custom_widget_manager()

drive.mount('/content/drive', force_remount=True)
DB_FILE = '/content/drive/MyDrive/Jhon/App/pass'


# ─────────────────────────────────────────────
# 3. Funciones auxiliares de CRUD
# ─────────────────────────────────────────────
def crear_registro(name: str, user:str, clave:str) -> int:
    sql: str = 'INSERT INTO pass(nombre, usuario, clave) VALUES(?, ?, ?)'

    try:
        with sqlite3.connect(DB_FILE) as conn:
            cursor = conn.cursor()
            cursor.execute(sql, (name, user, clave))
            conn.commit()

        return 0

    except sqlite3.Error as e:
        print(f"😞 Error de SQLite: {e}")
        return 2

# end


def seleccionar_datos() -> list[tuple]:
    sql = 'SELECT * FROM pass'

    # Conecta y asegura cierre automático
    try:
        with sqlite3.connect(DB_FILE) as conn:
            cursor = conn.cursor()
            cursor.execute(sql)
            resultados = cursor.fetchall()

        return resultados

    except sqlite3.Error as e:
        print(f"😞 Error de SQLite: {e}")
        return []

# end


def actualizar_registro(id: int, name: str, user:str, clave:str) -> int:
    # sql: str = 'UPDATE pass SET clave = ' +  "'" + parametro +  "'" + ' WHERE id = ' + id
    sql: str = 'UPDATE pass SET nombre = ?, usuario = ?, clave = ? WHERE id = ?'

    try:
        with sqlite3.connect(DB_FILE) as conn:
            cursor = conn.cursor()
            cursor.execute(sql, (name, user, clave, id))
            conn.commit()

        return 0

    except sqlite3.Error as e:
        print(f"😞 Error de SQLite: {e}")
        return 2

# end


def eliminar_registro(id: int) -> int:
    sql: str = 'DELETE FROM pass WHERE id = ' + str(id)

    try:
        with sqlite3.connect(DB_FILE) as conn:
            cursor = conn.cursor()
            cursor.execute(sql)
            conn.commit()

        return 0

    except sqlite3.Error as e:
        print(f"😞 Error de SQLite: {e}")
        return 2
    else:
        output.clear()

# end


# ─────────────────────────────────────────────
# 4. Widgets
# ─────────────────────────────────────────────

# Campos de entrada
txt_nombre = Text(description="Nombre:", placeholder="Nombre completo")
txt_usuario = Text(description="Usuario:", placeholder="Usuario")
txt_contrasena = Password(description="Contraseña:")


# Botones
btn_crear = Button(description="Crear", button_style="success")
btn_actualizar = Button(description="Actualizar", button_style="info")
btn_eliminar = Button(description="Eliminar", button_style="danger", disabled=True)
btn_refrescar = Button(description="Listar", button_style="")


# Dropdown para seleccionar registro (por ID)
ddl_registros = Dropdown(
    description="ID:",
    options=[],
    value=None
)

# Área de salida para mostrar mensajes y tabla
out_mensajes = Output()
out_lista = Output()


# ─────────────────────────────────────────────
# 5. Lógica para actualizar el Dropdown y la lista
# ─────────────────────────────────────────────
def refrescar_dropdown() -> None:
    filas = seleccionar_datos()
    opciones: list[tuple[str, None]] = []

    if not filas:
        ddl_registros.options = []
        ddl_registros.value = None
    else:
        # Mostrar como "ID - Nombre"
        opciones.append(('0 - Seleccionar', 0))
        for f in filas:
            opciones.append((f"{f[0]} - {f[1]}", f[0]))

        ddl_registros.options = opciones
        ddl_registros.value = opciones[0][1]

# end


def mostrar_lista() -> None:
    filas = seleccionar_datos()

    with out_lista:
        clear_output()
        if not filas:
            print("No hay registros en la base de datos.")
        else:
            print()
            print(f"{'ID':<5} {'Nombre':<25} {'Usuario':<35} {'Contraseña':<20}")
            print("-" * 85)
            for f in filas:
                id_, nombre, usuario, contrasena = f
                print(f"{id_:<5} {nombre:<25} {usuario:<35} {contrasena:<20}")

# end

# Inicializar dropdown
refrescar_dropdown()


# ─────────────────────────────────────────────
# 6. Callbacks de los botones
# ─────────────────────────────────────────────
def on_crear_clicked(b):
    nombre = txt_nombre.value.strip()
    usuario = txt_usuario.value.strip()
    contrasena = txt_contrasena.value.strip()

    with out_mensajes:
        clear_output()
        if not (nombre and usuario and contrasena):
            print("⚠️ Todos los campos son obligatorios.")
            return

        crear_registro(nombre, usuario, contrasena)
        print()
        print("✅ Registro creado correctamente.")

    # Limpiar campos
    txt_nombre.value = ""
    txt_usuario.value = ""
    txt_contrasena.value = ""

    # Actualizar lista y dropdown
    refrescar_dropdown()
    mostrar_lista()

# end


def on_actualizar_clicked(b):
    if ddl_registros.value is None:
        with out_mensajes:
            clear_output()
            print("⚠️ No hay registro seleccionado para actualizar.")
        return

    id_registro = ddl_registros.value
    nombre = txt_nombre.value.strip()
    usuario = txt_usuario.value.strip()
    contrasena = txt_contrasena.value.strip()

    with out_mensajes:
        clear_output()
        if not (nombre and usuario and contrasena):
            print("⚠️ Todos los campos son obligatorios para actualizar.")
            return

        actualizar_registro(id_registro, nombre, usuario, contrasena)
        print()
        print(f"✅ Registro con ID {id_registro} actualizado correctamente.")

    refrescar_dropdown()
    mostrar_lista()

# end


def on_eliminar_clicked(b):
    if ddl_registros.value is None:
        with out_mensajes:
            clear_output()
            print("⚠️ No hay registro seleccionado para eliminar.")
        return

    id_registro = ddl_registros.value
    eliminar_registro(id_registro)

    with out_mensajes:
        clear_output()
        print()
        print(f"🗑️ Registro con ID {id_registro} eliminado.")

    refrescar_dropdown()
    mostrar_lista()

# end


def on_refrescar_clicked(b):
    refrescar_dropdown()
    mostrar_lista()

    with out_mensajes:
        clear_output()
        print()
        print("🔄 Lista actualizada.")

# end


# Asociar callbacks
btn_crear.on_click(on_crear_clicked)
btn_actualizar.on_click(on_actualizar_clicked)
btn_eliminar.on_click(on_eliminar_clicked)
btn_refrescar.on_click(on_refrescar_clicked)


# ─────────────────────────────────────────────
# 7. Cargar datos del registro seleccionado en los campos
# ─────────────────────────────────────────────
def on_registro_cambiado(change) -> None:
    if change["new"] is None:
        return

    id_registro = change["new"]

    with sqlite3.connect(DB_FILE) as conn:
            cursor = conn.cursor()
            cursor.execute('SELECT nombre, usuario, clave FROM pass WHERE id = ?', (id_registro,))
            resultados = cursor.fetchall()

    if resultados:
        nombre, usuario, contrasena = resultados[0]
        txt_nombre.value = nombre
        txt_usuario.value = usuario
        txt_contrasena.value = contrasena

        # activar boton eliminar
        btn_eliminar.disabled = False
    else:
        # Limpiar campos
        txt_nombre.value = ""
        txt_usuario.value = ""
        txt_contrasena.value = ""

        # desactivar boton eliminar
        btn_eliminar.disabled = True

# end


ddl_registros.observe(on_registro_cambiado, names="value")

# ─────────────────────────────────────────────
# 8. Mostrar la interfaz
# ─────────────────────────────────────────────
box_campos = VBox([
    txt_nombre,
    txt_usuario,
    txt_contrasena
])

box_botones = HBox([
    btn_crear,
    btn_actualizar,
    btn_eliminar,
    btn_refrescar
])

ui = VBox([
    Label("CRUD básico con SQLITE"),
    ddl_registros,
    box_campos,
    box_botones,
    out_mensajes,
    out_lista
])

print()
display(ui)

Mounted at /content/drive

